<a href="https://colab.research.google.com/github/LadyLandshark/Annoyatron/blob/master/Assign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Numerical Algorithms

## Part 1: Numerical Inversion by Binary Search

In [54]:
# Utilities/Common
import random
import sys
import numpy as np
import scipy

from typing import Callable
random.seed(42) # Constant seed for debugging

EPSILON = np.finfo(float).eps # Machine epsilon for comparing floating point numbers
def comp_floats(a: float, b: float, threshold=EPSILON):
  delta = a - b
  if delta == np.nan:
    raise Error("Floats incomparable")

  #print(f"a: {a}, b: {b}, delta: {delta}")
  if abs(delta) < threshold:
    return 0
  else:
    return delta

In [89]:
# Part 1

def invert(func: Callable, yval, interval=None):
  bottom = 0
  top = random.random() + EPSILON # Never 0
  #top = np.inf
  if interval:
    bottom = interval[0]
    top = interval[1]
    if top < bottom:
      raise Error("Interval must be increasing")
  while(True):
    mid = bottom + (top - bottom)/2
    comp = comp_floats(yval, func(mid), 1e-10)
    print(f"bottom: {bottom}, mid: {mid}, top: {top}, comp: {comp}")
    if comp == 0:
      return mid
    elif comp == np.inf:
      bottom = mid
      top = sys.float_info.max
    elif comp > 0:
      bottom = mid
      top = (mid + comp)*2
    else:
      top = mid
    if comp_floats(bottom, top, 1e-10) == 0:
      return bottom


In [ ]:
# Part 1.1

def p1_test_func(x):
  return x * np.exp(x) + x
samples = [0] + \
 [random.random() for x in range(1, 11)]
# [sys.float_info.max] If yval is infinite there's probably lots of numbers that cause it

for x in samples:
  y = p1_test_func(x)
  x_gen = invert(p1_test_func, y)
  print(f"y: {y}, x: {x}, x_gen: {x_gen}")
  assert(comp_floats(x_gen, x, 1e-10) == 0)

In [95]:
# Part 1.2
def p1_2_test_func(x):
  return scipy.special.gamma(x)
samples = [1] + \
 [random.uniform(1, 10) for x in range(1, 11)] #+ \
 #[sys.float_info.max]

for x in samples:
  y = p1_2_test_func(x)
  x_gen = invert(p1_2_test_func, y, (1, 10))
  print(f"y: {y}, x: {x}, x_gen: {x_gen}")
  assert(comp_floats(x_gen, x, 1e-10) == 0)

bottom: 1, mid: 5.5, top: 10, comp: -51.34277778455352
bottom: 1, mid: 3.25, top: 5.5, comp: -1.549256966718529
bottom: 1, mid: 2.125, top: 3.25, comp: -0.059460537330914054
bottom: 1, mid: 1.5625, top: 2.125, comp: 0.11015561813493047
bottom: 1.5625, mid: 2.4539056181349306, top: 3.345311236269861, comp: -0.2876207396666828
bottom: 1.5625, mid: 2.0082028090674653, top: 2.4539056181349306, comp: -0.0034957756588402944
bottom: 1.5625, mid: 1.7853514045337326, top: 2.0082028090674653, comp: 0.07242279756653447
bottom: 1.7853514045337326, mid: 2.7504499043671338, top: 3.7155484042005344, comp: -0.6089521656792225
bottom: 1.7853514045337326, mid: 2.2679006544504334, top: 2.7504499043671338, comp: -0.14477679117975417
bottom: 1.7853514045337326, mid: 2.026626029492083, top: 2.2679006544504334, comp: -0.011550617699018995
bottom: 1.7853514045337326, mid: 1.9059887170129077, top: 2.026626029492083, comp: 0.03616857421956632
bottom: 1.9059887170129077, mid: 2.895151649738928, top: 3.8843145824

AssertionError: ignored

## Part 2: Numerical Integration

Using the Romberg method as I felt like not doing Monte Carlo.

In [52]:
def integrate(func: Callable, interval):
  max_steps = 5
  tolerance_abs = .05
  tolerance_rel = 0.01

  botm = interval[0]
  top = interval[1]

  step_size = abs(botm - top)
  buf = [[0 for x in range(max_steps+1)] for x in range(max_steps+1)]
  buf[0][0] = step_size/2 * (func(botm) + func(top))
  integral_total = buf[0][0]

  for i in range(1, max_steps+1):
    step_size /= 2
    integral_curr = 0
    for j in range(1, 2**i-1):
      integral_local += func(botm + step_size*j)
    buf[i][0] = buf[i-1][0]/2 + integral_curr * step_size;
    for k in range(1,i+1):
      additional = buf[i][k-1] - buf[i-1][k-1]
      additional /= 4**k - 1
      buf[i][k] = buf[i][k-1] + additional

      # Absolute approximate error
      abs_approx_err = abs(buf[i][k] - buf[i][k-1])
      # Absolute relative approximate error
      abs_rel_approx_err = abs(abs_approx_err/buf[i][k])

      integral_total = buf[i][k]
      # If we meet tolerance
      if abs_rel_approx_err < tolerance_rel or abs_approx_err < tolerance_abs:
        return integral_total
  # If we don't, return anyway
  return integral_total






In [ ]:
def p2_test_func(x):
  return np.sin(x)/x